In [ ]:
import pandas as pd
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD, DCTERMS, OWL

# --- Load Data
df = pd.read_csv("Baris Data - Sheet1 (2).csv")

# --- RDF Setup
g = Graph()
ONT = Namespace("http://yusupbanyuwangi.org/ontology#")
LEXINFO = Namespace("http://www.lexinfo.net/ontology/2.0/lexinfo#")

g.bind("", ONT)
g.bind("rdf", RDF)
g.bind("rdfs", RDFS)
g.bind("xsd", XSD)
g.bind("dcterms", DCTERMS)
g.bind("owl", OWL)
g.bind("lexinfo", LEXINFO)

# --- Instance Umum
manuskrip_uri = ONT["naskahYusup01"]
g.add((manuskrip_uri, RDF.type, ONT.Manuskrip))
g.add((manuskrip_uri, DCTERMS.title, Literal("Lontar Yusup Banyuwangi", lang="jv")))
g.add((manuskrip_uri, DCTERMS.language, Literal("jv")))
g.add((manuskrip_uri, ONT.ditulisMenggunakan, ONT.AksaraPegon))

g.add((ONT.AksaraPegon, RDF.type, ONT.SistemTulisan))
g.add((ONT.AksaraPegon, RDFS.label, Literal("Arab Pegon", lang="id")))

# --- Loop Per Baris
for index, row in df.iterrows():
    idx = f"{int(row['id_baris']):03d}"
    baris_uri = ONT[f"baris{idx}"]
    aksara_uri = ONT[f"aksara_baris{idx}"]
    translit_uri = ONT[f"translit_baris{idx}"]
    terjemah_uri = ONT[f"terjemah_baris{idx}"]

    g.add((baris_uri, RDF.type, ONT.BarisNaskah))
    g.add((baris_uri, ONT.isFromManuscript, manuskrip_uri))
    g.add((baris_uri, ONT.mengandungAksara, aksara_uri))
    g.add((baris_uri, ONT.hasTransliteration, translit_uri))
    g.add((baris_uri, ONT.hasTranslation, terjemah_uri))
    g.add((baris_uri, DCTERMS.identifier, Literal(f"YUSUP01-B{idx}")))

    g.add((aksara_uri, RDF.type, ONT.TeksAksara))
    g.add((aksara_uri, RDF.value, Literal(row['aksara'], lang="urlImage")))
    g.add((aksara_uri, LEXINFO.script, ONT.AksaraPegon))

    g.add((translit_uri, RDF.type, ONT.Transliterasi))
    g.add((translit_uri, RDF.value, Literal(row['transliterasi'], lang="jv-Latn")))
    g.add((translit_uri, ONT.menggunakanAturan, Literal("Transliterasi Arab Pegon Standar")))

    g.add((terjemah_uri, RDF.type, ONT.Terjemahan))
    g.add((terjemah_uri, RDF.value, Literal(row['terjemahan'], lang="id")))
    g.add((terjemah_uri, ONT.keBahasa, Literal("id")))
    g.add((terjemah_uri, ONT.dariBahasa, Literal("jv")))

# --- Simpan
g.serialize("output_lontar.ttl", format="turtle")